In [1]:
using Revise

In [117]:
using Plots; plotly()
using CoordinateTransformations
using ForwardDiff
using ColorTypes

In [27]:
import Nets
import LearningMPC

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [131]:
hidden_widths = [16, 16]
activation = Nets.leaky_relu
nx = 2
nu = 1
widths = [nx, hidden_widths..., nu]
x_to_u = AffineMap(eye(nx), zeros(nx))
v_to_y = AffineMap(diagm([20.]), zeros(nu))
params = 0.1 * randn(Nets.Params{Float64}, widths).data
net = Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y)

f(x) = [20 * sin(1 * x[1]) * sin(.4 * x[2])]
# f(x) = [x[1] + x[2]]

function f_sensitive(x)
    v = f(x)
    J = ForwardDiff.jacobian(f, x)
    hcat(v, J)
end

f_sensitive(randn(2))

training_data = Tuple{Vector{Float64}, Matrix{Float64}}[]
for i in 1:200
    x = 2 .* randn(2)
    push!(training_data, (x, f_sensitive(x)))
end

testing_data = Tuple{Vector{Float64}, Matrix{Float64}}[]
for i in 1:200
    x = 2 .* randn(2)
    push!(testing_data, (x, f_sensitive(x)))
end

training_loss(net) = average_loss(net, training_data)
testing_loss(net) = average_loss(net, testing_data)

average_loss(net, data) = mean(data) do xy
    x, uJ = xy
    sum(abs2, net(x) .- uJ[:, 1])
end


gradient_sensitivity = 0.5
learning_loss = LearningMPC.sensitive_loss(net, gradient_sensitivity)
adam_opts = Nets.AdamOpts(learning_rate=1e-2, batch_size=1)
optimizer = Nets.AdamOptimizer(learning_loss, adam_opts, net, zeros(2), zeros(1, 3))


all_losses(net) = (training_loss(net),
                   testing_loss(net),
                   mean(training_data) do xy
                       learning_loss(net.params.data, xy...)
                   end)

losses = Tuple{Float64, Float64, Float64}[]

 for i in 1:200
    Nets.update!(net.params.data, optimizer, training_data)
    optimizer.opts.learning_rate *= (1 - 1e-3)
#     Nets.adam_update!(net.params.data, updater, learning_loss, 
#         training_data, adam_opts);
    push!(losses, all_losses(net))
end

plt = plot(first.(losses), label="training", yscale=:log10)
plot!(plt, getindex.(losses, 2), label="testing")
plot!(plt, getindex.(losses, 3), label="learning")
ylims!(plt, (0.01, ylims(plt)[2]))

In [132]:
plt = plot([x[1] for (x, y) in training_data], [x[2] for (x, y) in training_data], [y[1] for (x, y) in training_data], 
    line=nothing, marker=:dot, markersize=1)

surface!(plt, linspace(-4, 4), linspace(-4, 4), (x, y) -> net([x, y])[1])
plot!(plt, [x[1] for (x, y) in testing_data], [x[2] for (x, y) in testing_data], [y[1] for (x, y) in testing_data], 
      line=nothing, marker=:dot, markersize=1, markercolor=colorant"red")


zlims!(plt, (-30, 30))
plt